# Tutorial 6-1: The Residual Revolution – "Building ResNet"

**Course:** CSEN 342: Deep Learning  
**Topic:** Residual Networks, Skip Connections, and Bottleneck Layers

## Objective
As we saw in the previous lecture, simply stacking more layers doesn't always lead to better performance. In fact, very deep "plain" networks (20+ layers) often have *higher* training error than shallower ones due to optimization difficulties.

The **ResNet** (Residual Network) architecture solved this by introducing the **Skip Connection**, allowing gradients to flow directly through the network.

In this tutorial, we will:
1.  **Implement the Residual Block:** Build the fundamental building block $H(x) = F(x) + x$.
2.  **Assemble ResNet:** Stack these blocks to create a deep network (ResNet-18 style) for CIFAR-10.
3.  **Implement the Bottleneck:** Build the advanced block used in ResNet-50/101/152 that uses 1x1 convolutions to reduce computation.

---

## Part 1: Setup and Data

We use the utility function to download CIFAR-10 safely on the cluster.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import os
import sys

# Add parent directory to path to import utils
sys.path.append(os.path.abspath(os.path.join('..')))
from utils import download_cifar10

# 1. Pre-download Data
download_cifar10()

# 2. Prepare DataLoaders
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

print(f"Data loaded on {device}")

---

## Part 2: The Basic Residual Block

The core idea is to learn a **residual function** $F(x)$ rather than the direct mapping $H(x)$. We do this by adding the input $x$ to the output of the convolutional layers.

**Structure:**
1.  Conv3x3 (preserve spatial size)
2.  BatchNorm + ReLU
3.  Conv3x3
4.  BatchNorm
5.  **Add Input (Skip Connection)**
6.  ReLU

**Challenge:** If we change the number of channels (e.g., 64 -> 128) or spatial size (stride 2), we can't just add $x$ because the shapes don't match. We need a **Shortcut Connection** (usually a 1x1 conv) to resize $x$[cite: 4030].

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1 # Used for ResNet sizing

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        # Main Path (F(x))
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        # Shortcut Path (x)
        self.shortcut = nn.Sequential()
        
        # If dimensions change (stride > 1 or channels increase), we need to adapt x
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        # F(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        # + x (The Skip Connection)
        out += self.shortcut(x)
        
        # Final ReLU
        out = F.relu(out)
        return out

---

## Part 3: The Bottleneck Block

For very deep networks (ResNet-50 and above), using two 3x3 convolutions becomes expensive. We use a **Bottleneck** design.

**Structure (Slide 52):**
1.  **1x1 Conv:** Reduce dimensions (squeeze).
2.  **3x3 Conv:** Process features in lower dimensions.
3.  **1x1 Conv:** Restore dimensions (expand).

This allows the network to be much deeper with similar computational cost.

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4 # Output channels are 4x input channels in the main path

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        # 1. Squeeze (1x1)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        
        # 2. Process (3x3)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        # 3. Expand (1x1)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        
        # Skip Connection
        out += self.shortcut(x)
        
        out = F.relu(out)
        return out

---

## Part 4: Building ResNet

Now we assemble the blocks into a full network. We follow the pattern of ResNet-18[cite: 4045]:
1.  **Stem:** Initial Conv + BatchNorm + ReLU.
2.  **Layers:** 4 groups of residual blocks, doubling the filters periodically.
3.  **Head:** Global Average Pooling + Fully Connected Layer.

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # Stem
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        # Layers (groups of blocks)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        # Head
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # Stem
        out = F.relu(self.bn1(self.conv1(x)))
        
        # Blocks
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        # Head (Global Average Pooling)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

model = ResNet18().to(device)
print("ResNet-18 Created.")
# Check parameter count
num_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {num_params:,}")

## Part 5: Training

We will train for a few epochs to verify convergence.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
    print(f"Epoch {epoch}: Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.2f}%")

print("Starting Training...")
for epoch in range(1, 4): # Train for 3 epochs for demo
    train(epoch)
    scheduler.step()

### Conclusion
You have successfully implemented a modern ResNet from scratch. Key takeaways:
1.  **Skip Connections** enable deep networks by creating a "highway" for gradients.
2.  **Basic Blocks** ($3\times3$) are used for smaller ResNets (18, 34).
3.  **Bottleneck Blocks** ($1\times1 \to 3\times3 \to 1\times1$) are used for deeper ResNets (50+) to save computation.